# Week 3 peer-graded assignment

In [22]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [138]:
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import numpy as np
import folium
from folium.plugins import HeatMap
from bs4 import BeautifulSoup
import selenium

print('Libraries Imported')

Libraries Imported


# Toronto Clustered

## 1. Download and Explore Dataset

### Canada data set

In [2]:
#web scraping 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text

soup = BeautifulSoup(html_data, 'html5lib')
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    
    if row.span.text =='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df_temp=pd.DataFrame(table_contents)
df_temp['Borough']=df_temp['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
#import Geospatial_Coordinates.csv
geo_temp = pd.read_csv('Geospatial_Coordinates.csv')
geo_temp.rename(columns = {'Postal Code':'PostalCode'} , inplace = True)

canada_df = pd.merge(df_temp,geo_temp, on=['PostalCode'])

In [3]:
canada_df.sample(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
44,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
22,M1G,Scarborough,Woburn,43.770992,-79.216917
78,M1S,Scarborough,Agincourt,43.794200,-79.262029
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
37,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750


### Select Borough Toronto

In [4]:
toronto_df = canada_df[canada_df['Borough'].str.contains('Toronto')].reset_index(drop =True)

In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[0]))

The dataframe has 7 boroughs and 39 neighborhoods.


#### function for get the latitude and longitude values 

In [6]:
def get_lat_long(address):
    address = address
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
    return latitude, longitude

#### Function create a map of location with neighborhoods superimposed on top.

In [18]:
def create_map (df, latitude, longitude):
    # create map of New York using latitude and longitude values
    map_location = folium.Map(location=[latitude, longitude], zoom_start=10)
      
    # add circle
    for lat, lng,borough  in zip(df['Latitude'], df['Longitude'],df['Borough']):
        folium.Circle(
        radius=1000,
        location=[lat, lng],
        popup=borough,
        color="crimson",
        fill=False,
        ).add_to(map_location)

    # add markers to map
    for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_location)
    return map_location

In [27]:
toronto_lat, toronto_long = get_lat_long('Toronto')
map_toronto = create_map(toronto_df, toronto_lat, toronto_long)
map_toronto.save("Fig1.html")
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## 2. Explore Neighborhoods in toronto

#### Define Foursquare Credentials and Version

In [29]:
CLIENT_ID = '5RIYSV55BA3TWBBUDIAH0ZHMHOPQ40DNVMCDYCOPE3AMTD4F' # your Foursquare ID
CLIENT_SECRET = 'EEQEVE41EZFU2HOH1BGGTGALVYQS15ZLHHAJ3Q5JAMSS2PJE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5RIYSV55BA3TWBBUDIAH0ZHMHOPQ40DNVMCDYCOPE3AMTD4F
CLIENT_SECRET:EEQEVE41EZFU2HOH1BGGTGALVYQS15ZLHHAJ3Q5JAMSS2PJE


#### function getNearby top venues that are in location  within radius

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
# type your answer here
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [32]:
toronto_venues['Venue Category'] = toronto_venues['Venue Category'].replace({'Neighborhood':'district'})

In [40]:
toronto_venues.sample(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
257,Berczy Park,43.644771,-79.373306,Market Street Catch,43.648501,-79.371808,Seafood Restaurant
297,Central Bay Street,43.657952,-79.387383,Bubble Bath & Spa,43.659050,-79.385344,Spa
736,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,DAVIDsTEA,43.650547,-79.383385,Tea Room
107,"Garden District, Ryerson",43.657162,-79.378937,Tangerine Café,43.653937,-79.379722,Bank
1331,Enclave of M5E,43.646435,-79.374846,Shoppers Drug Mart,43.647141,-79.370898,Pharmacy
563,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,World Café at Harbourfront Centre,43.638347,-79.383046,Café
1014,North Toronto West,43.715383,-79.405678,A&W,43.715149,-79.399944,Fast Food Restaurant
568,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Canadian Waters,43.642103,-79.386253,Aquarium
673,"The Danforth West, Riverdale",43.679557,-79.352188,Leonidas Chocolates Cafe,43.678118,-79.349485,Café
429,"Richmond, Adelaide, King",43.650571,-79.384568,Equinox Bay Street,43.648100,-79.379989,Gym


In [35]:
venue_count_by_neighborhood = toronto_venues[['Venue','Neighborhood']].groupby('Neighborhood').count()
venue_count_by_neighborhood.sample(5)

,Venue
Neighborhood,
Berczy Park,57
"The Annex, North Midtown, Yorkville",20
Studio District,36
"India Bazaar, The Beaches West",18
"Regent Park, Harbourfront",44


In [101]:
coffee_shop = toronto_venues[toronto_venues['Venue Category'].isin(['Coffee Shop','Café'])]

In [102]:
count_coffee_shop = coffee_shop[['Neighborhood','Venue Category']].groupby('Neighborhood',as_index=False).count()

In [135]:
test = pd.merge(count_coffee_shop, toronto_df , how='left' , on=['Neighborhood'])

In [140]:
test.head()

,Neighborhood,Count CoffeeShop&Cafe,PostalCode,Borough,Latitude,Longitude
0,Berczy Park,4,M5E,Downtown Toronto,43.644771,-79.373306
1,"Brockton, Parkdale Village, Exhibition Place",5,M6K,West Toronto,43.636847,-79.428191
2,"CN Tower, King and Spadina, Railway Lands, Har...",1,M5V,Downtown Toronto,43.628947,-79.394420
3,Central Bay Street,15,M5G,Downtown Toronto,43.657952,-79.387383
4,Christie,4,M6G,Downtown Toronto,43.669542,-79.422564


In [103]:
count_coffee_shop.columns = ['Neighborhood', 'Count CoffeeShop&Cafe']

In [106]:
temp = count_coffee_shop.sort_values(by='Count CoffeeShop&Cafe').reset_index(drop =True)

In [116]:
Neighborhood_list = temp.loc[0:20]['Neighborhood'].tolist()

In [117]:
Select_Neighborhood = toronto_df[toronto_df['Neighborhood'].isin(Neighborhood_list)]

In [130]:
def create_map (df, latitude, longitude):
    # create map of New York using latitude and longitude values
    map_location = folium.Map(location=[latitude, longitude], zoom_start=10)
    
    for lat, lng,borough  in zip(df['Latitude'], df['Longitude'], df['Borough']):
        folium.Circle(
        radius=300,
        location=[lat, lng],
        popup=borough,
        color="crimson",
        fill=False,
        ).add_to(map_location)
      
    # add markers to map
    for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
        label = '{}'.format(neighborhood)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=1,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_location)
    return map_location

In [ ]:
HeatMap(data=toronto_venues[['Latitude', 'Longitude', 'Count CoffeeShop&Cafe']], radius=8, max_zoom=13).add_to(coffee_map)

In [142]:
coffee_map = create_map(Select_Neighborhood, toronto_lat, toronto_long)
coffee_map 

#### Let's find out how many unique categories can be curated from all the returned venues

In [102]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


## 3. Analyze Each Neighborhood


In [144]:
Neighborhood_list

['CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport',
 'Dufferin, Dovercourt Village',
 'Parkdale, Roncesvalles',
 'Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park',
 'High Park, The Junction South',
 'North Toronto West',
 'Little Portugal, Trinity',
 'Berczy Park',
 'Christie',
 'Davisville',
 'Brockton, Parkdale Village, Exhibition Place',
 'The Danforth West, Riverdale',
 'Studio District',
 'St. James Town, Cabbagetown',
 'The Annex, North Midtown, Yorkville',
 'Runnymede, Swansea',
 'University of Toronto, Harbord',
 'Church and Wellesley',
 'Kensington Market, Chinatown, Grange Park',
 'Regent Park, Harbourfront',
 'St. James Town']

In [145]:
select_toronto_venues = toronto_venues[toronto_venues['Neighborhood'].isin(Neighborhood_list)]

In [147]:
select_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [148]:
# one hot encoding
toronto_onehot = pd.get_dummies(select_toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot = pd.concat([select_toronto_venues['Neighborhood'],toronto_onehot],axis = 1)

toronto_onehot.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio,district
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
toronto_onehot.shape

(770, 193)

In [160]:
toronto_grouped = toronto_onehot.groupby(('Neighborhood')).mean().reset_index()
toronto_grouped.sample(5)

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio,district
5,Davisville,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.0,...,0.0,0.027778,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
15,"St. James Town, Cabbagetown",0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
14,St. James Town,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.022727,0.0,...,0.0,0.000000,0.0,0.011364,0.000000,0.0,0.011364,0.0,0.000000,0.0
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.0625,0.1875,0.125,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
20,"University of Toronto, Harbord",0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.027778,0.0,0.000000,0.0,0.027778,0.0


In [161]:
toronto_grouped.shape

(21, 193)

In [152]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0        Bakery  0.05
1   Coffee Shop  0.05
2  Cocktail Bar  0.05
3      Pharmacy  0.04
4    Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.12
1                  Bakery  0.08
2          Breakfast Spot  0.08
3             Coffee Shop  0.08
4  Furniture / Home Store  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0     Airport Service  0.19
1    Airport Terminal  0.12
2         Coffee Shop  0.06
3  Airport Food Court  0.06
4        Airport Gate  0.06


----Christie----
                venue  freq
0       Grocery Store  0.25
1                Café  0.19
2                Park  0.12
3           Nightclub  0.06
4  Italian Restaurant  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1  Japanese Restaurant  0.06
2     Sushi

#### Let's put that into a _pandas_ dataframe

In [169]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [175]:
def put_most_venue_to_df (num_top_venues,location_grouped):
    num_top_venues = num_top_venues

    indicators = ['st', 'nd', 'rd']

    # create columns according to number of top venues
    columns = ['Neighborhood']
    for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))

    # create a new dataframe
    neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
    neighborhoods_venues_sorted['Neighborhood'] = location_grouped['Neighborhood']

    for ind in np.arange(location_grouped.shape[0]):
        neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(location_grouped.iloc[ind, :], num_top_venues)
    return neighborhoods_venues_sorted

In [189]:
toronto_venues_most = put_most_venue_to_df(10, toronto_grouped)
toronto_venues_most.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Bakery,Coffee Shop,Cocktail Bar,Pharmacy,Restaurant,Beer Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Gourmet Shop
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Furniture / Home Store,Stadium,Restaurant,Climbing Gym,Bar,Burrito Place
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Coffee Shop,Airport Food Court,Airport Gate,Airport Lounge,Plane,Airport,Bar,Rental Car Location
3,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Baby Store,Athletics & Sports,Restaurant,Candy Store,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Fast Food Restaurant,Yoga Studio,Hotel,Pub,Men's Store


## 4. Cluster Neighborhoods

Run _k_-means to cluster the neighborhood into 5 clusters.

In [190]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 3, 2, 0, 0, 4, 4, 4, 4])

In [191]:
# add clustering labels
toronto_venues_most.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_most.set_index('Neighborhood'),how='right', on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Bakery,Coffee Shop,Cocktail Bar,Pharmacy,Restaurant,Beer Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Gourmet Shop
14,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,4,Café,Bakery,Breakfast Spot,Coffee Shop,Furniture / Home Store,Stadium,Restaurant,Climbing Gym,Bar,Burrito Place
32,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,3,Airport Service,Airport Terminal,Coffee Shop,Airport Food Court,Airport Gate,Airport Lounge,Plane,Airport,Bar,Rental Car Location
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564,2,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Baby Store,Athletics & Sports,Restaurant,Candy Store,Coffee Shop
37,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Fast Food Restaurant,Yoga Studio,Hotel,Pub,Men's Store


In [195]:
toronto_merged.shape

(21, 16)

In [192]:
def create_cluster_map (location, latitude, longitude, kclusters):
    # create map
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

    # set color scheme for the clusters
    kclusters = kclusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(location['Latitude'], location['Longitude'], location['Neighborhood'], location['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
    return map_clusters 

In [193]:
latitude,longitude = get_lat_long('Toronto')
map_clusters = create_cluster_map(toronto_merged,latitude, longitude, kclusters)
map_clusters

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## 5. Examine Clusters

#### Cluster 1

In [196]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Berczy Park,Bakery,Coffee Shop,Cocktail Bar,Pharmacy,Restaurant,Beer Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Gourmet Shop
37,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Fast Food Restaurant,Yoga Studio,Hotel,Pub,Men's Store
26,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Gym,Café,Sushi Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Pharmacy
23,North Toronto West,Coffee Shop,Clothing Store,Park,Fast Food Restaurant,Spa,Café,Sporting Goods Shop,Mexican Restaurant,Salon / Barbershop,Restaurant
0,"Regent Park, Harbourfront",Coffee Shop,Pub,Park,Bakery,Theater,Breakfast Spot,Café,French Restaurant,Dessert Shop,Farmers Market
2,St. James Town,Coffee Shop,Café,Restaurant,Clothing Store,Cosmetics Shop,Cocktail Bar,Bakery,Beer Bar,Park,Department Store
35,"St. James Town, Cabbagetown",Coffee Shop,Convenience Store,Market,Chinese Restaurant,Restaurant,Pub,Pizza Place,Bakery,Café,Italian Restaurant
17,Studio District,Coffee Shop,Brewery,Café,Gastropub,American Restaurant,Bakery,district,Diner,Seafood Restaurant,Cheese Shop
31,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Sushi Restaurant,Supermarket,Bank,Fried Chicken Joint,Liquor Store,Pub,Bagel Shop,American Restaurant,Vietnamese Restaurant
12,"The Danforth West, Riverdale",Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Café,Bubble Tea Shop,Brewery,Frozen Yogurt Shop,Juice Bar


#### Cluster 2

In [197]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,"Parkdale, Roncesvalles",Gift Shop,Breakfast Spot,Italian Restaurant,Dessert Shop,Eastern European Restaurant,Movie Theater,Bar,Dog Run,Restaurant,Bookstore


#### Cluster 3

In [198]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Baby Store,Athletics & Sports,Restaurant,Candy Store,Coffee Shop


#### Cluster 4

In [199]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Coffee Shop,Airport Food Court,Airport Gate,Airport Lounge,Plane,Airport,Bar,Rental Car Location


#### Cluster 5

In [200]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Furniture / Home Store,Stadium,Restaurant,Climbing Gym,Bar,Burrito Place
8,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Portuguese Restaurant,Middle Eastern Restaurant,Music Venue,Park,Gym / Fitness Center,Café,Bus Stop,Smoke Shop
22,"High Park, The Junction South",Mexican Restaurant,Café,Thai Restaurant,Cajun / Creole Restaurant,Italian Restaurant,Bookstore,Furniture / Home Store,Music Venue,Bar,Bakery
30,"Kensington Market, Chinatown, Grange Park",Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Park,Caribbean Restaurant,Burger Joint,Dumpling Restaurant,Grocery Store
11,"Little Portugal, Trinity",Bar,Asian Restaurant,Diner,Men's Store,Vietnamese Restaurant,Restaurant,Café,Italian Restaurant,Coffee Shop,Japanese Restaurant
28,"Runnymede, Swansea",Café,Coffee Shop,Pizza Place,Sushi Restaurant,Italian Restaurant,Pub,Gourmet Shop,French Restaurant,Latin American Restaurant,Bookstore
24,"The Annex, North Midtown, Yorkville",Coffee Shop,Café,Sandwich Place,Burger Joint,Liquor Store,History Museum,Park,Donut Shop,Indian Restaurant,Pub
27,"University of Toronto, Harbord",Café,Bookstore,Theater,Bakery,Japanese Restaurant,Bar,Sandwich Place,Video Game Store,Dessert Shop,Restaurant
